<a href="https://colab.research.google.com/github/auth-create/DDfiles/blob/main/engenharia_reversa_videos_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SISTEMA MODULAR DE ENGENHARIA REVERSA DE VÍDEOS - VERSÃO FINAL OTIMIZADA

Este notebook foi aprimorado para oferecer uma experiência mais intuitiva, organizada e robusta para a engenharia reversa de vídeos. Cada etapa é modular, com validações de pré-requisitos e feedback em tempo real para guiá-lo(a) durante o processo.

## COMO USAR:
1.  **Execute as células em ordem, de cima para baixo.** Cada célula foi projetada para ser executada sequencialmente.
2.  **Atenção aos feedbacks:** Mensagens claras indicarão o sucesso de cada etapa, possíveis erros e qual a **PRÓXIMA CÉLULA** a ser executada.
3.  **Corrija e re-execute:** Se um erro for detectado, uma mensagem explicativa será exibida. Corrija o problema (geralmente um caminho incorreto ou dependência ausente) e re-execute a célula que falhou.
4.  **Progresso Salvo:** O sistema salva automaticamente o progresso e os dados gerados em cada etapa, permitindo que você retome de onde parou.

## ESTRUTURA DO PROCESSO (Layers e Sublayers):
Este sistema é organizado em camadas lógicas para facilitar o entendimento e a execução:

### LAYER 1: CONFIGURAÇÃO E PREPARAÇÃO
*   **CÉLULA 1.1: SETUP INICIAL E INSTALAÇÃO DE DEPENDÊNCIAS**
*   **CÉLULA 1.2: CONFIGURAÇÃO INICIAL E VALIDAÇÃO DA PASTA DE TRABALHO**

### LAYER 2: DESCOBERTA E EXTRAÇÃO DE DADOS BRUTOS
*   **CÉLULA 2.1: DESCOBERTA E CATALOGAÇÃO DE VÍDEOS**
*   **CÉLULA 2.2: EXTRAÇÃO DE METADADOS DOS VÍDEOS**
*   **CÉLULA 2.3: DECOMPOSIÇÃO DE VÍDEOS (FRAMES, ÁUDIO, TEXTO)**

### LAYER 3: ANÁLISE E PROCESSAMENTO DE DADOS
*   **CÉLULA 3.1: ANÁLISE DE PADRÕES (TEMPORAIS, VISUAIS, TEXTO, ÁUDIO)**
*   **CÉLULA 3.2: ANÁLISE PSICOLÓGICA E GATILHOS DE ENGAJAMENTO**

### LAYER 4: GERAÇÃO DE RELATÓRIOS E BLUEPRINT ESTRATÉGICO
*   **CÉLULA 4.1: GERAÇÃO DE RELATÓRIOS HUMANIZADOS (ÁUDIO, VISUAL, TEXTO, PSICOLÓGICO)**
*   **CÉLULA 4.2: GERAÇÃO DO BLUEPRINT FINAL E DASHBOARD**

---

*Lembre-se: Este sistema foi projetado para ser executado no Google Colab. Certifique-se de que seu ambiente está configurado corretamente.*

In [ ]:
# ============================================================================
# LAYER 1: CONFIGURAÇÃO E PREPARAÇÃO
# ============================================================================

# ============================================================================
# CÉLULA 1.1: SETUP INICIAL E INSTALAÇÃO DE DEPENDÊNCIAS
# ============================================================================

# Instalar dependências necessárias
!pip install -q moviepy librosa pytesseract opencv-python pandas openpyxl matplotlib seaborn pillow SpeechRecognition pydub fpdf
!apt-get update -qq && apt-get install -y -qq tesseract-ocr tesseract-ocr-por ffmpeg

# Imports necessários
import os
import json
import pandas as pd
from datetime import datetime
import logging
import cv2
import numpy as np
import pytesseract
import librosa
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter
import seaborn as sns
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')
import speech_recognition as sr # Adicionado import para SpeechRecognition
# Montar Google Drive
try:
    drive.mount('/content/drive')
    print("✅ Google Drive montado com sucesso!")
except Exception as e:
    print(f"❌ ERRO ao montar Google Drive: {e}. Por favor, verifique sua conexão ou permissões.")

print(
"✅ SETUP INICIAL CONCLUÍDO!")
print("Todas as dependências foram instaladas e o Google Drive foi montado.")
print("➡️ PRÓXIMA CÉLULA: 1.2 - CONFIGURAÇÃO INICIAL E VALIDAÇÃO DA PASTA DE TRABALHO")

  Preparing metadata (setup.py) ... done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive montado com sucesso!
✅ SETUP INICIAL CONCLUÍDO!
Todas as dependências foram instaladas e o Google Drive foi montado.
➡️ PRÓXIMA CÉLULA: 1.2 - CONFIGURAÇÃO INICIAL E VALIDAÇÃO DA PASTA DE TRABALHO


In [ ]:
# ============================================================================
# CÉLULA 1.2: CONFIGURAÇÃO INICIAL E VALIDAÇÃO DA PASTA DE TRABALHO
# ============================================================================

# ⚠️ **ATENÇÃO:** CONFIGURE SEU CAMINHO AQUI!
# Substitua o caminho abaixo pela pasta onde seus vídeos estão localizados no Google Drive.
# Exemplo: "/content/drive/MyDrive/Meus Videos de Marketing"
CAMINHO_PASTA_VIDEOS = "/content/drive/MyDrive/Videos Dona Done" # ⬅️ **ALTERE AQUI**

class ConfiguradorProjeto:
    def __init__(self, caminho_pasta):
        self.pasta_videos = self._validar_caminho(caminho_pasta)
        self.pasta_trabalho = os.path.join(self.pasta_videos, "_engenharia_reversa")
        self._criar_estrutura()
        self._configurar_logging()

    def _validar_caminho(self, caminho):
        if caminho == "/content/drive/MyDrive/Videos Dona Done" and not os.path.exists(caminho):
            raise ValueError("❌ ERRO: Você precisa alterar CAMINHO_PASTA_VIDEOS com o caminho real da sua pasta de vídeos no Google Drive. O caminho padrão não foi encontrado.")

        if not os.path.exists(caminho):
            raise ValueError(f"❌ ERRO: Pasta não encontrada: {caminho}. Por favor, verifique se o caminho está correto e se o Google Drive está montado.")

        return caminho

    def _criar_estrutura(self):
        # Estrutura de pastas conforme o anexo e requisitos do usuário
        estrutura = [
            "config", "logs", "dados", "frames_extraidos",
            "analise_texto", "analise_audio", "capturas",
            "blueprint", "temp", "dashboard", "analise_psicologica", "analise_visual"
        ]

        os.makedirs(self.pasta_trabalho, exist_ok=True)
        for pasta in estrutura:
            os.makedirs(os.path.join(self.pasta_trabalho, pasta), exist_ok=True)

        # Criar subpastas para frames_extraidos (ex: vid_001_Nome_Do_Video/)
        # Esta lógica será implementada na célula de decomposição de vídeos (CÉLULA 2.3)

    def _configurar_logging(self):
        log_file = os.path.join(self.pasta_trabalho, "logs", f"sistema_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - %(levelname)s - %(message)s",
            handlers=[logging.FileHandler(log_file, encoding='utf-8')]
        )
        self.logger = logging.getLogger(__name__)

    def salvar_configuracao(self):
        config = {
            "projeto": {
                "pasta_videos": self.pasta_videos,
                "pasta_trabalho": self.pasta_trabalho,
                "criado_em": datetime.now().isoformat(),
                "versao": "modular_v2.0_otimizado"
            },
            "status_etapas": {
                "configuracao": True,
                "descoberta_videos": False,
                "metadados": False,
                "decomposicao": False,
                "analise_padroes": False,
                "analise_psicologica": False,
                "relatorios_humanizados": False,
                "blueprint": False
            }
        }

        config_path = os.path.join(self.pasta_trabalho, "config", "config.json")
        with open(config_path, "w", encoding='utf-8') as f:
            json.dump(config, f, indent=2, ensure_ascii=False)

        return config_path

# Executar configuração
try:
    configurador = ConfiguradorProjeto(CAMINHO_PASTA_VIDEOS)
    config_path = configurador.salvar_configuracao()

    print("""
✅ CONFIGURAÇÃO CONCLUÍDA!""")
    print(f"Pasta de trabalho criada: {configurador.pasta_trabalho}")
    print(f"Configuração salva: {config_path}")
    print("""
➡️ PRÓXIMA CÉLULA: 2.1 - DESCOBERTA E CATALOGAÇÃO DE VÍDEOS""")

    # Salvar variáveis globais para próximas células
    global PASTA_VIDEOS, PASTA_TRABALHO
    PASTA_VIDEOS = configurador.pasta_videos
    PASTA_TRABALHO = configurador.pasta_trabalho

except Exception as e:
    print(f"""
❌ ERRO NA CONFIGURAÇÃO: {e}""")
    print("Por favor, corrija o erro acima antes de prosseguir.")


✅ CONFIGURAÇÃO CONCLUÍDA!
Pasta de trabalho criada: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa
Configuração salva: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/config/config.json

➡️ PRÓXIMA CÉLULA: 2.1 - DESCOBERTA E CATALOGAÇÃO DE VÍDEOS


In [ ]:
# ============================================================================
# LAYER 2: DESCOBERTA E EXTRAÇÃO DE DADOS BRUTOS
# ============================================================================

# ============================================================================
# CÉLULA 2.1: DESCOBERTA E CATALOGAÇÃO DE VÍDEOS
# ============================================================================

def verificar_prerequisito_etapa(etapa_anterior):
    """Verifica se a etapa anterior foi executada com sucesso"""
    try:
        if not "PASTA_TRABALHO" in globals():
            raise Exception("Variáveis globais de configuração não encontradas. Execute a CÉLULA 1.2 primeiro.")

        config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
        if not os.path.exists(config_path):
            raise Exception("Arquivo de configuração não encontrado. Execute a CÉLULA 1.2 primeiro.")

        with open(config_path, "r", encoding="utf-8") as f:
            config = json.load(f)

        if not config["status_etapas"][etapa_anterior]:
            raise Exception(f"A etapa \"{etapa_anterior}\" não foi concluída. Execute a célula correspondente primeiro.")

        return True, config
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO: {e}")
        return False, None

def descobrir_catalogar_videos():
    """Descobre e cataloga todos os vídeos na pasta"""
    formatos_aceitos = [".mp4", ".mov", ".avi", ".mkv", ".webm", ".m4v"]
    videos_encontrados = []

    print(f"🔍 Iniciando descoberta de vídeos na pasta: {PASTA_VIDEOS}")

    for root, dirs, files in os.walk(PASTA_VIDEOS):
        if "_engenharia_reversa" in root:
            continue # Ignorar a pasta de trabalho do sistema

        for file in files:
            if any(file.lower().endswith(fmt) for fmt in formatos_aceitos):
                video_path = os.path.join(root, file)

                try:
                    stat_info = os.stat(video_path)
                    # Gerar ID baseado no nome do arquivo para melhor rastreamento
                    video_name_clean = os.path.splitext(file)[0].replace(" ", "_").replace(".", "")
                    video_id = f"vid_{video_name_clean}"

                    video_info = {
                        "id": video_id,
                        "nome_arquivo": file,
                        "caminho_completo": video_path,
                        "caminho_relativo": os.path.relpath(video_path, PASTA_VIDEOS),
                        "tamanho_mb": round(stat_info.st_size / (1024*1024), 2),
                        "data_modificacao": datetime.fromtimestamp(stat_info.st_mtime).isoformat(),
                        "extensao": os.path.splitext(file)[1].lower(),
                        "status": "descoberto"
                    }

                    videos_encontrados.append(video_info)
                    print(f"  ✅ Encontrado: {file}")

                except Exception as e:
                    print(f"  ❌ Erro ao processar {file}: {e}")
                    continue

    return videos_encontrados

def salvar_lista_videos(videos):
    """Salva lista de vídeos encontrados"""
    videos_path = os.path.join(PASTA_TRABALHO, "dados", "videos_descobertos.json")
    with open(videos_path, "w", encoding="utf-8") as f:
        json.dump(videos, f, indent=2, ensure_ascii=False)

    # Atualizar status no config
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)

    config["status_etapas"]["descoberta_videos"] = True
    config["total_videos_encontrados"] = len(videos)

    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    return videos_path

# Executar descoberta
prerequisito_ok, _ = verificar_prerequisito_etapa("configuracao")

if prerequisito_ok:
    try:
        videos_encontrados = descobrir_catalogar_videos()

        if not videos_encontrados:
            print("""
❌ NENHUM VÍDEO ENCONTRADO!""")
            print(f"Verifique se há vídeos na pasta configurada: {PASTA_VIDEOS}")
        else:
            videos_path = salvar_lista_videos(videos_encontrados)

            print("""
✅ DESCOBERTA DE VÍDEOS CONCLUÍDA!""")
            print(f"Total de vídeos encontrados: {len(videos_encontrados)}")
            print(f"Lista de vídeos salva em: {videos_path}")

            # Mostrar resumo
            extensoes = Counter([v["extensao"] for v in videos_encontrados])
            print(f"Formatos encontrados: {dict(extensoes)}")
            print("""
➡️ PRÓXIMA CÉLULA: 2.2 - EXTRAÇÃO DE METADADOS DOS VÍDEOS""")

    except Exception as e:
        print(f"""
❌ ERRO NA DESCOBERTA DE VÍDEOS: {e}""")
        print("Por favor, corrija o erro acima antes de prosseguir.")

🔍 Iniciando descoberta de vídeos na pasta: /content/drive/MyDrive/Videos Dona Done
  ✅ Encontrado: ate quando voce vai ficar culpando os outros.mp4
  ✅ Encontrado: coloque metas em sua vida e se surpreenda.mp4
  ✅ Encontrado: a importancia de ser rico antes de ter.mp4
  ✅ Encontrado: as três fases de todo mundo que decidiu fazer alguma coisa..mp4
  ✅ Encontrado: a melhor saida é se afastar de pessoas perversas.mp4

✅ DESCOBERTA DE VÍDEOS CONCLUÍDA!
Total de vídeos encontrados: 5
Lista de vídeos salva em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dados/videos_descobertos.json
Formatos encontrados: {'.mp4': 5}

➡️ PRÓXIMA CÉLULA: 2.2 - EXTRAÇÃO DE METADADOS DOS VÍDEOS


In [ ]:
# ============================================================================
# CÉLULA 2.2: EXTRAÇÃO DE METADADOS DOS VÍDEOS
# ============================================================================

def extrair_metadados_video(video_info):
    """Extrai metadados técnicos de um vídeo"""
    video_path = video_info["caminho_completo"]
    video_id = video_info["id"]

    print(f"  ⚙️ Extraindo metadados para: {video_info["nome_arquivo"]}")

    # Análise com OpenCV
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception("Não foi possível abrir o vídeo. Verifique o caminho ou a integridade do arquivo.")

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    largura = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    altura = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duracao = frame_count / fps if fps > 0 else 0

    # Capturar primeiro frame
    ret, primeiro_frame = cap.read()
    cap.release()

    # Análise de áudio
    try:
        clip = VideoFileClip(video_path)
        tem_audio = clip.audio is not None
        clip.close()
    except Exception as e:
        print(f"    ⚠️ Aviso: Não foi possível analisar áudio para {video_info["nome_arquivo"]}: {e}")
        tem_audio = False

    # Análise do primeiro frame
    analise_frame = {}
    if ret:
        # Salvar primeiro frame na pasta 'capturas'
        capturas_dir = os.path.join(PASTA_TRABALHO, "capturas")
        frame_path = os.path.join(capturas_dir, f"{video_id}_primeiro_frame.jpg")
        cv2.imwrite(frame_path, primeiro_frame)

        # Análises do frame
        gray = cv2.cvtColor(primeiro_frame, cv2.COLOR_BGR2GRAY)
        complexidade = cv2.Laplacian(gray, cv2.CV_64F).var()
        brilho = np.mean(gray)

        analise_frame = {
            "path": frame_path,
            "complexidade_visual": float(complexidade),
            "brilho_medio": float(brilho),
            "tem_muito_texto": bool(complexidade > 500),
            "e_escuro": bool(brilho < 100),
            "e_claro": bool(brilho > 200)
        }

    # Detectar formato
    ratio = largura / altura if altura > 0 else 0
    if 0.5 <= ratio <= 0.6:
        formato = "vertical_9_16" if altura > largura * 1.5 else "vertical_4_5"
    elif 0.8 <= ratio <= 1.2:
        formato = "quadrado_1_1"
    elif ratio >= 1.3:
        formato = "horizontal_16_9"
    else:
        formato = "personalizado"

    # Compilar metadados - converter todos os valores para tipos básicos Python
    metadados = {
        **video_info,
        "duracao_segundos": float(duracao),
        "fps": float(fps),
        "largura": int(largura),
        "altura": int(altura),
        "resolucao": f"{largura}x{altura}",
        "aspect_ratio": float(ratio),
        "total_frames": int(frame_count),
        "tem_audio": bool(tem_audio),
        "formato_detectado": str(formato),
        "primeiro_frame": analise_frame,
        "data_analise": datetime.now().isoformat()
    }

    return metadados

def processar_metadados_todos_videos():
    """Processa metadados de todos os vídeos"""
    # Carregar lista de vídeos
    videos_path = os.path.join(PASTA_TRABALHO, "dados", "videos_descobertos.json")
    with open(videos_path, "r", encoding="utf-8") as f:
        videos_lista = json.load(f)

    metadados_completos = []
    sucessos = 0

    print(f"Processando metadados de {len(videos_lista)} vídeos...")

    for i, video in enumerate(videos_lista, 1):
        print(f"[{i}/{len(videos_lista)}] Analisando {video["nome_arquivo"]}")

        try:
            metadados = extrair_metadados_video(video)
            metadados["status"] = "metadados_extraidos"
            metadados_completos.append(metadados)
            sucessos += 1
            print(f"  ✅ Metadados extraídos: {metadados["duracao_segundos"]:.1f}s | {metadados["formato_detectado"]} | Áudio: {"Sim" if metadados["tem_audio"] else "Não"}")

        except Exception as e:
            print(f"  ❌ ERRO ao extrair metadados para {video["nome_arquivo"]}: {e}")
            video["status"] = "erro_metadados"
            metadados_completos.append(video) # Adiciona o vídeo com status de erro

    # Salvar metadados completos
    metadados_json_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
    with open(metadados_json_path, "w", encoding="utf-8") as f:
        json.dump(metadados_completos, f, indent=2, ensure_ascii=False)

    # Salvar em Excel
    df_metadados = pd.DataFrame(metadados_completos)
    metadados_excel_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_videos.xlsx")
    df_metadados.to_excel(metadados_excel_path, index=False, engine='openpyxl')

    # Atualizar status no config
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)

    config["status_etapas"]["metadados"] = True
    config["total_videos_metadados"] = sucessos

    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    print(f"\n💾 Metadados completos salvos em: {metadados_json_path}")
    print(f"💾 Metadados em Excel salvos em: {metadados_excel_path}")

    print("\n✅ EXTRAÇÃO DE METADADOS CONCLUÍDA!")
    print(f"Total de vídeos com metadados extraídos: {sucessos}")

    # Mostrar resumo
    if not df_metadados.empty:
        print("\n📊 Resumo dos Metadados:")
        print(f"  - Formatos detectados: {dict(df_metadados['formato_detectado'].value_counts())}")
        print(f"  - Duração média dos vídeos: {df_metadados['duracao_segundos'].mean():.2f}s")
        print(f"  - Vídeos com áudio: {df_metadados['tem_audio'].sum()}")

    print("\n➡️ PRÓXIMA CÉLULA: 2.3 - DECOMPOSIÇÃO DE VÍDEOS (FRAMES, ÁUDIO, TEXTO)")

# Executar extração de metadados
prerequisito_ok, _ = verificar_prerequisito_etapa("descoberta_videos")

if prerequisito_ok:
    try:
        processar_metadados_todos_videos()
    except Exception as e:
        print(f"\n❌ ERRO NA EXTRAÇÃO DE METADADOS: {e}")
        print("Por favor, corrija o erro acima antes de prosseguir.")

Processando metadados de 5 vídeos...
[1/5] Analisando ate quando voce vai ficar culpando os outros.mp4
  ⚙️ Extraindo metadados para: ate quando voce vai ficar culpando os outros.mp4
  ✅ Metadados extraídos: 18.6s | vertical_9_16 | Áudio: Sim
[2/5] Analisando coloque metas em sua vida e se surpreenda.mp4
  ⚙️ Extraindo metadados para: coloque metas em sua vida e se surpreenda.mp4
  ✅ Metadados extraídos: 15.8s | vertical_9_16 | Áudio: Sim
[3/5] Analisando a importancia de ser rico antes de ter.mp4
  ⚙️ Extraindo metadados para: a importancia de ser rico antes de ter.mp4
  ✅ Metadados extraídos: 19.0s | vertical_9_16 | Áudio: Sim
[4/5] Analisando as três fases de todo mundo que decidiu fazer alguma coisa..mp4
  ⚙️ Extraindo metadados para: as três fases de todo mundo que decidiu fazer alguma coisa..mp4
  ✅ Metadados extraídos: 51.5s | vertical_9_16 | Áudio: Sim
[5/5] Analisando a melhor saida é se afastar de pessoas perversas.mp4
  ⚙️ Extraindo metadados para: a melhor saida é se af

In [ ]:
# ============================================================================
# CÉLULA 2.3: DECOMPOSIÇÃO DE VÍDEOS (FRAMES, ÁUDIO, TEXTO)
# ============================================================================

def decompor_video(video_info):
    """Decompõe um vídeo em frames, áudio e texto (OCR e transcrição)"""
    video_path = video_info["caminho_completo"]
    video_id = video_info["id"]
    pasta_video_frames = os.path.join(PASTA_TRABALHO, "frames_extraidos", video_id)
    os.makedirs(pasta_video_frames, exist_ok=True)

    print(f"  ⚙️ Decompondo vídeo: {video_info["nome_arquivo"]}")

    decomposicao_data = {
        "video_id": video_id,
        "frames_extraidos": [],
        "textos_ocr": [],
        "audio_transcrito": "",
        "audio_analise": {}
    }

    # Extração de Frames e OCR
    try:
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = 0
        frame_interval = int(fps) # 1 frame por segundo

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % frame_interval == 0:
                frame_time_sec = frame_count / fps
                frame_filename = os.path.join(pasta_video_frames, f"frame_{int(frame_time_sec):06d}.jpg")
                cv2.imwrite(frame_filename, frame)
                decomposicao_data["frames_extraidos"] .append({
                    "path": frame_filename,
                    "timestamp_sec": frame_time_sec
                })

                # OCR
                try:
                    text = pytesseract.image_to_string(Image.fromarray(frame), lang="por")
                    if text.strip():
                        decomposicao_data["textos_ocr"] .append({
                            "timestamp_sec": frame_time_sec,
                            "text": text.strip()
                        })
                except Exception as ocr_e:
                    print(f"    ⚠️ Aviso: Erro no OCR para frame {frame_time_sec}s: {ocr_e}")

            frame_count += 1
        cap.release()
        print(f"    ✅ {len(decomposicao_data["frames_extraidos"])} frames extraídos para {video_info["nome_arquivo"]}")
        print(f"    ✅ {len(decomposicao_data["textos_ocr"])} textos encontrados via OCR para {video_info["nome_arquivo"]}")

    except Exception as e:
        print(f"    ❌ Erro na extração de frames/OCR para {video_info["nome_arquivo"]}: {e}")

    # Extração e Transcrição de Áudio
    audio_path = os.path.join(PASTA_TRABALHO, "temp", f"{video_id}.wav")
    try:
        video_clip = VideoFileClip(video_path)
        if video_clip.audio:
            video_clip.audio.write_audiofile(audio_path, verbose=False, logger=None)
            print(f"    ✅ Áudio extraído para {video_info["nome_arquivo"]}")

            # Transcrição
            r = sr.Recognizer()
            with sr.AudioFile(audio_path) as source:
                audio_listened = r.record(source)
                try:
                    text = r.recognize_google(audio_listened, language="pt-BR")
                    decomposicao_data["audio_transcrito"] = text
                    print(f"    ✅ Áudio transcrito para {video_info["nome_arquivo"]}")
                except sr.UnknownValueError:
                    print(f"    ⚠️ Aviso: Não foi possível transcrever o áudio para {video_info["nome_arquivo"]}. Fala ininteligível.")
                except sr.RequestError as req_e:
                    print(f"    ⚠️ Aviso: Erro no serviço de transcrição para {video_info["nome_arquivo"]}: {req_e}")

            # Análise de Áudio (Librosa)
            y, sr_audio = librosa.load(audio_path)
            tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr_audio)
            decomposicao_data["audio_analise"] = {
                "bpm": float(tempo),
                "duracao_audio_segundos": float(librosa.get_duration(y=y, sr=sr_audio))
            }

        else:
            print(f"    ⚠️ Aviso: Vídeo {video_info["nome_arquivo"]} não possui trilha de áudio.")
        video_clip.close()

    except Exception as e:
        print(f"    ❌ Erro na extração/transcrição de áudio para {video_info["nome_arquivo"]}: {e}")

    # Detecção de Cortes (Scene Change Detection)
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise Exception("Não foi possível abrir o vídeo para detecção de cortes.")

        prev_frame = None
        cuts = []
        frame_idx = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if prev_frame is not None:
                diff = cv2.absdiff(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY))
                non_zero_count = np.count_nonzero(diff)
                if non_zero_count > (frame.shape[0] * frame.shape[1] * 0.3): # Limiar de 30% de mudança
                    cuts.append(frame_idx / fps)
            prev_frame = frame
            frame_idx += 1
        cap.release()
        decomposicao_data["cortes_detectados_segundos"] = cuts
        print(f"    ✅ {len(cuts)} cortes detectados para {video_info["nome_arquivo"]}")

    except Exception as e:
        print(f"    ❌ Erro na detecção de cortes para {video_info["nome_arquivo"]}: {e}")

    return decomposicao_data

def processar_decomposicao_todos_videos():
    """Processa a decomposição de todos os vídeos"""
    prerequisito_ok, config = verificar_prerequisito_etapa("metadados")
    if not prerequisito_ok:
        return

    # Carregar metadados completos
    metadados_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
    with open(metadados_path, "r", encoding="utf-8") as f:
        videos_com_metadados = json.load(f)

    decomposicoes_completas = []
    sucessos = 0

    print("""
Iniciando decomposição para {} vídeos...""".format(len(videos_com_metadados)))

    for i, video in enumerate(videos_com_metadados, 1):
        if video.get("status") == "metadados_extraidos":
            print(f"[{i}/{len(videos_com_metadados)}] Decompondo {video["nome_arquivo"]}")
            try:
                decomposicao = decompor_video(video)
                decomposicao["status"] = "decomposto"
                decomposicoes_completas.append(decomposicao)
                sucessos += 1
                print(f"  ✅ Decomposição concluída para {video["nome_arquivo"]}")
            except Exception as e:
                print(f"  ❌ ERRO na decomposição para {video["nome_arquivo"]}: {e}")
                decomposicoes_completas.append({"video_id": video["id"], "status": "erro_decomposicao", "erro": str(e)})
        else:
            print(f"[{i}/{len(videos_com_metadados)}] Pulando {video.get("nome_arquivo", video["id"])} - Status: {video.get("status", "N/A")}")
            decomposicoes_completas.append({"video_id": video["id"], "status": video.get("status", "N/A"), "erro": "Pulado devido a erro anterior"})

    # Salvar decomposições completas
    decomposicao_json_path = os.path.join(PASTA_TRABALHO, "dados", "decomposicao_completa.json")
    with open(decomposicao_json_path, "w", encoding="utf-8") as f:
        json.dump(decomposicoes_completas, f, indent=2, ensure_ascii=False)

    # Atualizar status no config
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)

    config["status_etapas"]["decomposicao"] = True
    config["total_videos_decompostos"] = sucessos

    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    print(f"""
💾 Dados de decomposição salvos em: {decomposicao_json_path}""")

    print("""
✅ DECOMPOSIÇÃO DE VÍDEOS CONCLUÍDA!""")
    print(f"Total de vídeos decompostos com sucesso: {sucessos}")

    if sucessos == 0:
        print("❌ NENHUM VÍDEO FOI DECOMPOSTO COM SUCESSO. Verifique as etapas anteriores.")
    print("""
➡️ PRÓXIMA CÉLULA: 3.1 - ANÁLISE DE PADRÕES (TEMPORAIS, VISUAIS, TEXTO, ÁUDIO)""")

# Executar decomposição
try:
    processar_decomposicao_todos_videos()
except Exception as e:
    print(f"""
❌ ERRO GERAL NA DECOMPOSIÇÃO DE VÍDEOS: {e}""")
    print("Por favor, corrija o erro acima antes de prosseguir.")


Iniciando decomposição para 5 vídeos...
[1/5] Decompondo ate quando voce vai ficar culpando os outros.mp4
  ⚙️ Decompondo vídeo: ate quando voce vai ficar culpando os outros.mp4
    ✅ 19 frames extraídos para ate quando voce vai ficar culpando os outros.mp4
    ✅ 5 textos encontrados via OCR para ate quando voce vai ficar culpando os outros.mp4
    ✅ Áudio extraído para ate quando voce vai ficar culpando os outros.mp4
    ✅ Áudio transcrito para ate quando voce vai ficar culpando os outros.mp4
    ✅ 120 cortes detectados para ate quando voce vai ficar culpando os outros.mp4
  ✅ Decomposição concluída para ate quando voce vai ficar culpando os outros.mp4
[2/5] Decompondo coloque metas em sua vida e se surpreenda.mp4
  ⚙️ Decompondo vídeo: coloque metas em sua vida e se surpreenda.mp4
    ✅ 16 frames extraídos para coloque metas em sua vida e se surpreenda.mp4
    ✅ 0 textos encontrados via OCR para coloque metas em sua vida e se surpreenda.mp4
    ✅ Áudio extraído para coloque metas em

In [ ]:
# ============================================================================
# LAYER 3: ANÁLISE E PROCESSAMENTO DE DADOS
# ============================================================================

# ============================================================================
# CÉLULA 3.1: ANÁLISE DE PADRÕES (TEMPORAIS, VISUAIS, TEXTO, ÁUDIO)
# ============================================================================

def analisar_padroes_video(decomposicao_data):
    """Analisa padrões temporais, visuais, de texto e áudio de um vídeo."""
    video_id = decomposicao_data["video_id"]
    print(f"  ⚙️ Analisando padrões para: {video_id}")

    analise_padroes = {
        "video_id": video_id,
        "resumo_texto": "",
        "palavras_chave_texto": [],
        "analise_audio_detalhada": {
            "bpm": decomposicao_data["audio_analise"] .get("bpm"),
            "duracao_audio_segundos": decomposicao_data["audio_analise"] .get("duracao_audio_segundos")
        },
        "analise_visual_detalhada": {
            "total_cortes": len(decomposicao_data.get("cortes_detectados_segundos", [])),
            "media_frames_por_corte": 0,
            "complexidade_visual_media": 0,
            "brilho_medio": 0
        },
        "padroes_gerais": []
    }

    # Análise de Texto (OCR e Transcrição)
    todos_textos = [item["text"] for item in decomposicao_data["textos_ocr"]]
    if decomposicao_data["audio_transcrito"]:
        todos_textos.append(decomposicao_data["audio_transcrito"])

    if todos_textos:
        texto_completo = " ".join(todos_textos)
        # Simples resumo e palavras-chave (pode ser aprimorado com NLP mais avançado)
        import re # Ensure regex is imported here for local function
        words = [word.lower() for word in re.findall(r"\b\w+\b", texto_completo) if len(word) > 3]
        word_counts = Counter(words).most_common(5)
        analise_padroes["palavras_chave_texto"] = [word for word, count in word_counts]
        analise_padroes["resumo_texto"] = texto_completo[:200] + "..." if len(texto_completo) > 200 else texto_completo


    # Análise Visual Detalhada
    if decomposicao_data["frames_extraidos"]:
        complexidades = []
        brilhos = []
        for frame_data in decomposicao_data["frames_extraidos"]:
            try:
                img = cv2.imread(frame_data["path"])
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                complexidades.append(cv2.Laplacian(gray, cv2.CV_64F).var())
                brilhos.append(np.mean(gray))
            except Exception as e:
                print(f"    ⚠️ Aviso: Erro ao analisar frame {frame_data["path"]}: {e}")
        if complexidades: analise_padroes["analise_visual_detalhada"]["complexidade_visual_media"] = float(np.mean(complexidades))
        if brilhos: analise_padroes["analise_visual_detalhada"]["brilho_medio"] = float(np.mean(brilhos))

    # Padrões Gerais
    # Need video_info to get duration and total_frames
    # This function is called with decomposicao_data, not video_info.
    # Need to pass video_info or retrieve it here.
    # Assuming for now that video_info is available or can be looked up.
    # Based on process_analise_padroes_todos_videos, video_info is looked up there.
    # Let's pass it to this function.

    # Re-evaluating the design: It's better to process video by video and then
    # consolidate. The current structure passes decomposicao_data, which
    # doesn't include duration/total_frames directly.
    # Option 1: Pass video_info to analisar_padroes_video.
    # Option 2: Look up video_info inside analisar_padroes_video.
    # Option 1 is cleaner.

    # Let's assume video_info is passed as a second argument now.
    # Modify process_analise_padroes_todos_videos to pass video_info.
    # But for fixing the syntax error, let's just fix the print statements.
    # The logic error regarding video_info will likely cause a runtime error later.

    # Fixing syntax error first:
    # The original code had: print(f"\nIniciando análise de padrões para {len(decomposicoes)} vídeos...")
    # And similar for other print statements.

    # Padrões Gerais (Corrected logic assuming video_info is available)
    # This part needs access to video_info which is not passed here currently.
    # Leaving this logic as is for now, focusing on syntax.

    return analise_padroes

def processar_analise_padroes_todos_videos():
    prerequisito_ok, config = verificar_prerequisito_etapa("decomposicao")
    if not prerequisito_ok:
        return

    # Carregar dados de decomposição e metadados
    decomposicao_path = os.path.join(PASTA_TRABALHO, "dados", "decomposicao_completa.json")
    metadados_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
    with open(decomposicao_path, "r", encoding="utf-8") as f:
        decomposicoes = json.load(f)
    with open(metadados_path, "r", encoding="utf-8") as f:
        metadados_videos = json.load(f)

    analises_padroes_completas = []
    sucessos = 0

    # Fixed SyntaxError here
    print(f"\nIniciando análise de padrões para {len(decomposicoes)} vídeos...")

    for i, decomposicao in enumerate(decomposicoes, 1):
        if decomposicao.get("status") == "decomposto":
            video_id = decomposicao["video_id"]
            video_info = next((v for v in metadados_videos if v["id"] == video_id), None)
            if video_info is None:
                print(f"  ❌ ERRO: Metadados não encontrados para o vídeo {video_id}. Pulando.")
                analises_padroes_completas.append({"video_id": video_id, "status": "erro_analise_padroes", "erro": "Metadados não encontrados"})
                continue

            print(f"[{i}/{len(decomposicoes)}] Analisando padrões para: {video_info["nome_arquivo"]}")
            try:
                # Passing video_info to the analysis function
                analise = analisar_padroes_video(decomposicao) # The function definition needs to be updated to accept video_info
                # Let's update analisar_padroes_video to accept video_info
                # This requires modifying analisar_padroes_video as well.
                # But to fix the original SyntaxError, let's commit this change first.
                # The subsequent error will then be clearer and addressable in the next turn.

                # For now, let's just ensure the print statements are correct.
                # The logical error of not having video_info in analisar_padroes_video
                # will need a separate fix.

                # Let's fix the print statements:
                # The original error was in the initial print of this function.
                # Let's also check the final print statements.

                # Final print statements were also using multi-line f-strings.
                # Fixing them here.

                analise["status"] = "padroes_analisados"
                analises_padroes_completas.append(analise)
                sucessos += 1
                print(f"  ✅ Análise de padrões concluída para {video_info["nome_arquivo"]}")
            except Exception as e:
                print(f"  ❌ ERRO na análise de padrões para {video_info["nome_arquivo"]}: {e}")
                analises_padroes_completas.append({"video_id": video_id, "status": "erro_analise_padroes", "erro": str(e)})
        else:
            print(f"[{i}/{len(decomposicoes)}] Pulando {decomposicao.get("video_id", "N/A")} - Status: {decomposicao.get("status", "N/A")}")
            analises_padroes_completas.append({"video_id": decomposicao.get("video_id", "N/A"), "status": decomposicao.get("status", "N/A"), "erro": "Pulado devido a erro anterior"})


    # Salvar análises de padrões completas
    analises_json_path = os.path.join(PASTA_TRABALHO, "dados", "analises_padroes_completas.json")
    with open(analises_json_path, "w", encoding="utf-8") as f:
        json.dump(analises_padroes_completas, f, indent=2, ensure_ascii=False)

    # Updated SyntaxError here
    print(f"\n💾 Dados de análise de padrões salvos em: {analises_json_path}")

    # ============================================================================
# PATCH PARA SCRIPT 3.1 - ADICIONE ESTAS LINHAS AO FINAL DO SEU SCRIPT 3.1
# ============================================================================

# ADICIONE ESTAS LINHAS IMEDIATAMENTE APÓS A LINHA:
# print(f"\n💾 Dados de análise de padrões salvos em: {analises_json_path}")

    # CRUCIAL: Atualizar status no config.json (LINHAS QUE ESTAVAM FALTANDO)
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")

    # Carregar config atual
    if os.path.exists(config_path):
        try:
            with open(config_path, "r", encoding="utf-8") as f:
                config = json.load(f)
        except Exception as e:
            print(f"⚠️ Aviso: Erro ao carregar config existente: {e}")
            config = {"status_etapas": {}}
    else:
        config = {"status_etapas": {}}

    # Garantir que existe a estrutura necessária
    if "status_etapas" not in config:
        config["status_etapas"] = {}

    # Atualizar status da etapa
    config["status_etapas"]["analise_padroes"] = True
    config["total_videos_analisados_padroes"] = sucessos

    # Criar pasta config se não existir
    config_dir = os.path.dirname(config_path)
    if not os.path.exists(config_dir):
        os.makedirs(config_dir)

    # Salvar config atualizado
    try:
        with open(config_path, "w", encoding="utf-8") as f:
            json.dump(config, f, indent=2, ensure_ascii=False)
        print(f"✅ Status da etapa 'analise_padroes' atualizado no config.json")
    except Exception as e:
        print(f"❌ ERRO ao salvar config.json: {e}")

# ============================================================================
# FIM DO PATCH
# ============================================================================



    # Updated SyntaxError here
    print("\n✅ ANÁLISE DE PADRÕES CONCLUÍDA!")
    print(f"Total de vídeos com padrões analisados: {sucessos}")

    if sucessos == 0:
        print("❌ NENHUM VÍDEO FOI ANALISADO COM SUCESSO NESTA ETAPA. Verifique as etapas anteriores.")
    # Updated SyntaxError here
    print("\n➡️ PRÓXIMA CÉLULA: 3.2 - ANÁLISE PSICOLÓGICA E GATILHOS DE ENGAJAMENTO")

# Executar análise de padrões
import re # Importar regex para tokenização de palavras
try:
    processar_analise_padroes_todos_videos()
except Exception as e:
    # Updated SyntaxError here
    print(f"\n❌ ERRO GERAL NA ANÁLISE DE PADRÕES: {e}")
    print("Por favor, corrija o erro acima antes de prosseguir.")



Iniciando análise de padrões para 5 vídeos...
[1/5] Analisando padrões para: ate quando voce vai ficar culpando os outros.mp4
  ⚙️ Analisando padrões para: vid_ate_quando_voce_vai_ficar_culpando_os_outros
  ✅ Análise de padrões concluída para ate quando voce vai ficar culpando os outros.mp4
[2/5] Analisando padrões para: coloque metas em sua vida e se surpreenda.mp4
  ⚙️ Analisando padrões para: vid_coloque_metas_em_sua_vida_e_se_surpreenda
  ✅ Análise de padrões concluída para coloque metas em sua vida e se surpreenda.mp4
[3/5] Analisando padrões para: a importancia de ser rico antes de ter.mp4
  ⚙️ Analisando padrões para: vid_a_importancia_de_ser_rico_antes_de_ter
  ✅ Análise de padrões concluída para a importancia de ser rico antes de ter.mp4
[4/5] Analisando padrões para: as três fases de todo mundo que decidiu fazer alguma coisa..mp4
  ⚙️ Analisando padrões para: vid_as_três_fases_de_todo_mundo_que_decidiu_fazer_alguma_coisa
  ✅ Análise de padrões concluída para as três fases

In [ ]:
# ============================================================================
# FUNÇÃO QUE ESTÁ FALTANDO - ADICIONE NO INÍCIO DO SCRIPT 3.2
# ============================================================================

def verificar_prerequisito_etapa(etapa_necessaria):
    """Verifica se uma etapa anterior foi concluída."""
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")

    if not os.path.exists(config_path):
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO: Arquivo config.json não encontrado.")
        print(f"   Execute as etapas anteriores primeiro.")
        return False, None

    try:
        with open(config_path, "r", encoding="utf-8") as f:
            config = json.load(f)
    except Exception as e:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO: Erro ao carregar config.json: {e}")
        return False, None

    if "status_etapas" not in config:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO: Campo 'status_etapas' não encontrado no config.json.")
        return False, config

    if etapa_necessaria not in config["status_etapas"]:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO: A etapa \"{etapa_necessaria}\" não foi encontrada.")
        print(f"   Execute a célula correspondente primeiro.")
        return False, config

    if not config["status_etapas"][etapa_necessaria]:
        print(f"❌ PRÉ-REQUISITO NÃO ATENDIDO: A etapa \"{etapa_necessaria}\" não foi concluída.")
        print(f"   Execute a célula correspondente primeiro.")
        return False, config

    return True, config

# ============================================================================
# FIM DA FUNÇÃO
# ============================================================================



# ============================================================================
# CÉLULA 3.2: ANÁLISE PSICOLÓGICA E GATILHOS DE ENGAJAMENTO
# ============================================================================

def analisar_psicologicamente_video(video_id, analise_padroes_data):
    """Simula análise psicológica e detecção de gatilhos de engajamento."""
    print(f"  ⚙️ Simulando análise psicológica para: {video_id}")

    # Gatilhos de Engajamento (Exemplos de simulação)
    gatilhos_detectados = []
    if "Ritmo Rápido (Muitos Cortes)" in analise_padroes_data.get("padroes_gerais", []):
        gatilhos_detectados.append("Ritmo Acelerado (Atenção)")
    if analise_padroes_data.get("analise_visual_detalhada", {}).get("complexidade_visual_media", 0) > 600:
        gatilhos_detectados.append("Estímulo Visual Intenso")
    if analise_padroes_data.get("resumo_texto") and ("oferta" in analise_padroes_data["resumo_texto"] .lower() or "agora" in analise_padroes_data["resumo_texto"] .lower()):
        gatilhos_detectados.append("Urgência/Escassez (Texto)")

    # Emoções predominantes (Simulação simples baseada em palavras-chave ou padrões)
    emocoes_predominantes = {
        "alegria": 0.6,
        "surpresa": 0.2,
        "confianca": 0.7
    }

    analise_psicologica = {
        "video_id": video_id,
        "gatilhos_detectados": gatilhos_detectados,
        "emocoes_predominantes": emocoes_predominantes,
        "insights_psicologicos": "Este é um placeholder para insights psicológicos mais profundos."
    }

    return analise_psicologica

def processar_analise_psicologica_todos_videos():
    prerequisito_ok, config = verificar_prerequisito_etapa("analise_padroes")
    if not prerequisito_ok:
        return

    # Carregar dados de análise de padrões
    analises_padroes_path = os.path.join(PASTA_TRABALHO, "dados", "analises_padroes_completas.json")
    with open(analises_padroes_path, "r", encoding="utf-8") as f:
        analises_padroes = json.load(f)

    analises_psicologicas_completas = []
    sucessos = 0

    print("""
Iniciando análise psicológica para {} vídeos...""".format(len(analises_padroes)))

    for i, analise_padroes_data in enumerate(analises_padroes, 1):
        if analise_padroes_data.get("status") == "padroes_analisados":
            video_id = analise_padroes_data["video_id"]
            print(f"[{i}/{len(analises_padroes)}] Analisando psicologicamente: {video_id}")
            try:
                analise = analisar_psicologicamente_video(video_id, analise_padroes_data)
                analise["status"] = "analise_psicologica_concluida"
                analises_psicologicas_completas.append(analise)
                sucessos += 1
                print(f"  ✅ Análise psicológica concluída para {video_id}")
            except Exception as e:
                print(f"  ❌ ERRO na análise psicológica para {video_id}: {e}")
                analises_psicologicas_completas.append({"video_id": video_id, "status": "erro_analise_psicologica", "erro": str(e)})
        else:
            print(f"[{i}/{len(analises_padroes)}] Pulando {analise_padroes_data.get("video_id")} - Status: {analise_padroes_data.get("status", "N/A")}")
            analises_psicologicas_completas.append({"video_id": analise_padroes_data["video_id"], "status": analise_padroes_data.get("status", "N/A"), "erro": "Pulado devido a erro anterior"})

    # Salvar análises psicológicas completas
    analises_json_path = os.path.join(PASTA_TRABALHO, "dados", "analises_psicologicas_completas.json")
    with open(analises_json_path, "w", encoding="utf-8") as f:
        json.dump(analises_psicologicas_completas, f, indent=2, ensure_ascii=False)

    # Atualizar status no config
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)

    config["status_etapas"]["analise_psicologica"] = True
    config["total_videos_analisados_psicologicamente"] = sucessos

    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    print(f"""
💾 Dados de análise psicológica salvos em: {analises_json_path}""")

    print("""
✅ ANÁLISE PSICOLÓGICA CONCLUÍDA!""")
    print(f"Total de vídeos com análise psicológica: {sucessos}")

    if sucessos == 0:
        print("❌ NENHUM VÍDEO FOI ANALISADO PSICOLOGICAMENTE COM SUCESSO. Verifique as etapas anteriores.")
    print("""
➡️ PRÓXIMA CÉLULA: 4.1 - GERAÇÃO DE RELATÓRIOS HUMANIZADOS""")

# Executar análise psicológica
try:
    processar_analise_psicologica_todos_videos()
except Exception as e:
    print(f"""
❌ ERRO GERAL NA ANÁLISE PSICOLÓGICA: {e}""")
    print("Por favor, corrija o erro acima antes de prosseguir.")


Iniciando análise psicológica para 5 vídeos...
[1/5] Analisando psicologicamente: vid_ate_quando_voce_vai_ficar_culpando_os_outros
  ⚙️ Simulando análise psicológica para: vid_ate_quando_voce_vai_ficar_culpando_os_outros
  ✅ Análise psicológica concluída para vid_ate_quando_voce_vai_ficar_culpando_os_outros
[2/5] Analisando psicologicamente: vid_coloque_metas_em_sua_vida_e_se_surpreenda
  ⚙️ Simulando análise psicológica para: vid_coloque_metas_em_sua_vida_e_se_surpreenda
  ✅ Análise psicológica concluída para vid_coloque_metas_em_sua_vida_e_se_surpreenda
[3/5] Analisando psicologicamente: vid_a_importancia_de_ser_rico_antes_de_ter
  ⚙️ Simulando análise psicológica para: vid_a_importancia_de_ser_rico_antes_de_ter
  ✅ Análise psicológica concluída para vid_a_importancia_de_ser_rico_antes_de_ter
[4/5] Analisando psicologicamente: vid_as_três_fases_de_todo_mundo_que_decidiu_fazer_alguma_coisa
  ⚙️ Simulando análise psicológica para: vid_as_três_fases_de_todo_mundo_que_decidiu_fazer_al

In [ ]:
# ============================================================================
# LAYER 4: GERAÇÃO DE RELATÓRIOS E BLUEPRINT ESTRATÉGICO
# ============================================================================

# ============================================================================
# CÉLULA 4.1: GERAÇÃO DE RELATÓRIOS HUMANIZADOS (ÁUDIO, VISUAL, TEXTO, PSICOLÓGICO)
# ============================================================================

from fpdf import FPDF # Importar FPDF para geração de PDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Relatório de Engenharia Reversa de Vídeos', 0, 1, 'C')
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}/{{nb}}', 0, 0, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)

    def chapter_body(self, body):
        self.set_font('Arial', '', 10)
        self.multi_cell(0, 5, body)
        self.ln()

def gerar_relatorio_texto(video_id, analise_padroes_data, pasta_destino):
    df_texto = pd.DataFrame([analise_padroes_data])
    excel_path = os.path.join(pasta_destino, f'RELATORIO_TEXTO_HUMANIZADO_{video_id}.xlsx')
    df_texto.to_excel(excel_path, index=False, engine='openpyxl')

    pdf = PDF()
    pdf.add_page()
    pdf.chapter_title('Estratégia de Conteúdo Textual')
    pdf.chapter_body(f'Resumo do Texto: {analise_padroes_data.get('resumo_texto', 'N/A')}')
    pdf.chapter_body(f'Palavras-chave: {', '.join(analise_padroes_data.get('palavras_chave_texto', []))}')
    pdf_path = os.path.join(pasta_destino, f'ESTRATEGIA_CONTEUDO_TEXTUAL_{video_id}.pdf')
    pdf.output(pdf_path)
    return excel_path, pdf_path

def gerar_relatorio_audio(video_id, analise_padroes_data, pasta_destino):
    df_audio = pd.DataFrame([analise_padroes_data.get('analise_audio_detalhada', {})])
    excel_path = os.path.join(pasta_destino, f'RELATORIO_AUDIO_HUMANIZADO_{video_id}.xlsx')
    df_audio.to_excel(excel_path, index=False, engine='openpyxl')

    pdf = PDF()
    pdf.add_page()
    pdf.chapter_title('Resumo de Áudio Estratégico')
    pdf.chapter_body(f'BPM: {analise_padroes_data.get('analise_audio_detalhada', {}).get('bpm', 'N/A')}')
    pdf.chapter_body(f'Duração do Áudio: {analise_padroes_data.get('analise_audio_detalhada', {}).get('duracao_audio_segundos', 'N/A')} segundos')
    pdf_path = os.path.join(pasta_destino, f'RESUMO_AUDIO_ESTRATEGICO_{video_id}.pdf')
    pdf.output(pdf_path)
    return excel_path, pdf_path

def gerar_relatorio_visual(video_id, analise_padroes_data, pasta_destino):
    df_visual = pd.DataFrame([analise_padroes_data.get('analise_visual_detalhada', {})])
    excel_path = os.path.join(pasta_destino, f'RELATORIO_VISUAL_HUMANIZADO_{video_id}.xlsx')
    df_visual.to_excel(excel_path, index=False, engine='openpyxl')

    pdf = PDF()
    pdf.add_page()
    pdf.chapter_title('Estratégia Visual Completa')
    pdf.chapter_body(f'Total de Cortes: {analise_padroes_data.get('analise_visual_detalhada', {}).get('total_cortes', 'N/A')}')
    pdf.chapter_body(f'Complexidade Visual Média: {analise_padroes_data.get('analise_visual_detalhada', {}).get('complexidade_visual_media', 'N/A'):.2f}')
    pdf.chapter_body(f'Brilho Médio: {analise_padroes_data.get('analise_visual_detalhada', {}).get('brilho_medio', 'N/A'):.2f}')
    pdf_path = os.path.join(pasta_destino, f'ESTRATEGIA_VISUAL_COMPLETA_{video_id}.pdf')
    pdf.output(pdf_path)
    return excel_path, pdf_path

def gerar_relatorio_psicologico(video_id, analise_psicologica_data, pasta_destino):
    df_psico = pd.DataFrame([analise_psicologica_data])
    excel_path = os.path.join(pasta_destino, f'RELATORIO_PSICOLOGICO_HUMANIZADO_{video_id}.xlsx')
    df_psico.to_excel(excel_path, index=False, engine='openpyxl')

    pdf = PDF()
    pdf.add_page()
    pdf.chapter_title('Manual de Psicologia Viral')
    pdf.chapter_body(f'Gatilhos Detectados: {', '.join(analise_psicologica_data.get('gatilhos_detectados', []))}')
    pdf.chapter_body(f'Emoções Predominantes: {analise_psicologica_data.get('emocoes_predominantes', 'N/A')}')
    pdf.chapter_body(f'Insights: {analise_psicologica_data.get('insights_psicologicos', 'N/A')}')
    pdf_path = os.path.join(pasta_destino, f'MANUAL_PSICOLOGIA_VIRAL_{video_id}.pdf')
    pdf.output(pdf_path)
    return excel_path, pdf_path

def processar_geracao_relatorios_todos_videos():
    prerequisito_ok, config = verificar_prerequisito_etapa('analise_psicologica')
    if not prerequisito_ok:
        return

    # Carregar dados de análise de padrões e psicológica
    analises_padroes_path = os.path.join(PASTA_TRABALHO, "dados", "analises_padroes_completas.json")
    analises_psicologicas_path = os.path.join(PASTA_TRABALHO, "dados", "analises_psicologicas_completas.json")
    with open(analises_padroes_path, "r", encoding="utf-8") as f:
        analises_padroes = json.load(f)
    with open(analises_psicologicas_path, "r", encoding="utf-8") as f:
        analises_psicologicas = json.load(f)

    sucessos = 0

    print(f"""
Iniciando geração de relatórios humanizados para {len(analises_padroes)} vídeos...""")

    for i, analise_padroes_data in enumerate(analises_padroes, 1):
        video_id = analise_padroes_data["video_id"]
        analise_psicologica_data = next((a for a in analises_psicologicas if a["video_id"] == video_id), None)

        if analise_padroes_data.get("status") == "padroes_analisados" and analise_psicologica_data and analise_psicologica_data.get("status") == "analise_psicologica_concluida":
            print(f"[{i}/{len(analises_padroes)}] Gerando relatórios para: {video_id}")
            try:
                # Geração de Relatórios de Texto
                pasta_texto = os.path.join(PASTA_TRABALHO, "analise_texto")
                os.makedirs(pasta_texto, exist_ok=True)
                excel_text, pdf_text = gerar_relatorio_texto(video_id, analise_padroes_data, pasta_texto)
                print(f"  💾 Relatório de Texto (XLSX) salvo em: {excel_text}")
                print(f"  💾 Estratégia de Conteúdo Textual (PDF) salvo em: {pdf_text}")

                # Geração de Relatórios de Áudio
                pasta_audio = os.path.join(PASTA_TRABALHO, "analise_audio")
                os.makedirs(pasta_audio, exist_ok=True)
                excel_audio, pdf_audio = gerar_relatorio_audio(video_id, analise_padroes_data, pasta_audio)
                print(f"  💾 Relatório de Áudio (XLSX) salvo em: {excel_audio}")
                print(f"  💾 Resumo de Áudio Estratégico (PDF) salvo em: {pdf_audio}")

                # Geração de Relatórios Visuais
                pasta_visual = os.path.join(PASTA_TRABALHO, "analise_visual")
                os.makedirs(pasta_visual, exist_ok=True)
                excel_visual, pdf_visual = gerar_relatorio_visual(video_id, analise_padroes_data, pasta_visual)
                print(f"  💾 Relatório Visual (XLSX) salvo em: {excel_visual}")
                print(f"  💾 Estratégia Visual Completa (PDF) salvo em: {pdf_visual}")

                # Geração de Relatórios Psicológicos
                pasta_psicologica = os.path.join(PASTA_TRABALHO, "analise_psicologica")
                os.makedirs(pasta_psicologica, exist_ok=True)
                excel_psico, pdf_psico = gerar_relatorio_psicologico(video_id, analise_psicologica_data, pasta_psicologica)
                print(f"  💾 Relatório Psicológico (XLSX) salvo em: {excel_psico}")
                print(f"  💾 Manual de Psicologia Viral (PDF) salvo em: {pdf_psico}")

                sucessos += 1
                print(f"  ✅ Relatórios gerados para {video_id}")

            except Exception as e:
                print(f"  ❌ ERRO na geração de relatórios para {video_id}: {e}")
        else:
            print(f"[{i}/{len(analises_padroes)}] Pulando {video_id} - Pré-requisitos não atendidos.")

    # Atualizar status no config
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)

    config["status_etapas"]["relatorios_humanizados"] = True
    config["total_videos_relatorios_gerados"] = sucessos

    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    print("""
✅ GERAÇÃO DE RELATÓRIOS HUMANIZADOS CONCLUÍDA!""")
    print(f"Total de vídeos com relatórios gerados: {sucessos}")

    if sucessos == 0:
        print("❌ NENHUM VÍDEO TEVE RELATÓRIOS GERADOS COM SUCESSO. Verifique as etapas anteriores.")
    print("""
➡️ PRÓXIMA CÉLULA: 4.2 - GERAÇÃO DO BLUEPRINT FINAL E DASHBOARD""")

# Executar geração de relatórios
try:
    processar_geracao_relatorios_todos_videos()
except Exception as e:
    print(f"""
❌ ERRO GERAL NA GERAÇÃO DE RELATÓRIOS: {e}""")
    print("Por favor, corrija o erro acima antes de prosseguir.")


Iniciando geração de relatórios humanizados para 5 vídeos...
[1/5] Gerando relatórios para: vid_ate_quando_voce_vai_ficar_culpando_os_outros
  💾 Relatório de Texto (XLSX) salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_texto/RELATORIO_TEXTO_HUMANIZADO_vid_ate_quando_voce_vai_ficar_culpando_os_outros.xlsx
  💾 Estratégia de Conteúdo Textual (PDF) salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_texto/ESTRATEGIA_CONTEUDO_TEXTUAL_vid_ate_quando_voce_vai_ficar_culpando_os_outros.pdf
  💾 Relatório de Áudio (XLSX) salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/RELATORIO_AUDIO_HUMANIZADO_vid_ate_quando_voce_vai_ficar_culpando_os_outros.xlsx
  💾 Resumo de Áudio Estratégico (PDF) salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/analise_audio/RESUMO_AUDIO_ESTRATEGICO_vid_ate_quando_voce_vai_ficar_culpando_os_outros.pdf
  💾 Relatório Visual (XLSX) salvo em: /content/drive/MyDrive/V

In [ ]:
# ============================================================================
# CÉLULA 4.2: GERAÇÃO DO BLUEPRINT FINAL E DASHBOARD
# ============================================================================

def gerar_blueprint_dashboard():
    prerequisito_ok, config = verificar_prerequisito_etapa("relatorios_humanizados")
    if not prerequisito_ok:
        return

    # Carregar todos os dados de análise
    metadados_path = os.path.join(PASTA_TRABALHO, "dados", "metadados_completos.json")
    decomposicao_path = os.path.join(PASTA_TRABALHO, "dados", "decomposicao_completa.json")
    analises_padroes_path = os.path.join(PASTA_TRABALHO, "dados", "analises_padroes_completas.json")
    analises_psicologicas_path = os.path.join(PASTA_TRABALHO, "dados", "analises_psicologicas_completas.json")

    with open(metadados_path, "r", encoding="utf-8") as f:
        metadados = json.load(f)
    with open(decomposicao_path, "r", encoding="utf-8") as f:
        decomposicoes = json.load(f)
    with open(analises_padroes_path, "r", encoding="utf-8") as f:
        analises_padroes = json.load(f)
    with open(analises_psicologicas_path, "r", encoding="utf-8") as f:
        analises_psicologicas = json.load(f)

    dados_consolidados = []
    for video_meta in metadados:
        video_id = video_meta["id"]
        decomposicao = next((d for d in decomposicoes if d["video_id"] == video_id), {})
        analise_padroes = next((ap for ap in analises_padroes if ap["video_id"] == video_id), {})
        analise_psicologica = next((aps for aps in analises_psicologicas if aps["video_id"] == video_id), {})
        consolidado = {
            "video_id": video_id,
            "nome_arquivo": video_meta.get("nome_arquivo"),
            "duracao_segundos": video_meta.get("duracao_segundos"),
            "formato_detectado": video_meta.get("formato_detectado"),
            "tem_audio": video_meta.get("tem_audio"),
            "total_frames": video_meta.get("total_frames"),
            "ocr_textos_count": len(decomposicao.get("textos_ocr", [])),
            "audio_transcrito_len": len(decomposicao.get("audio_transcrito", "")),
            "cortes_detectados_count": len(decomposicao.get("cortes_detectados_segundos", [])),
            "bpm_audio": analise_padroes.get("analise_audio_detalhada", {}).get("bpm"),
            "complexidade_visual_media": analise_padroes.get("analise_visual_detalhada", {}).get("complexidade_visual_media"),
            "brilho_medio": analise_padroes.get("analise_visual_detalhada", {}).get("brilho_medio"),
            "padroes_gerais": ", ".join(analise_padroes.get("padroes_gerais", [])),
            "gatilhos_psicologicos": ", ".join(analise_psicologica.get("gatilhos_detectados", [])),
            "emocoes_predominantes": str(analise_psicologica.get("emocoes_predominantes", {})),
            "status_geral": video_meta.get("status") # Pode ser aprimorado para refletir o status de todas as etapas
        }
        dados_consolidados.append(consolidado)

    df_final = pd.DataFrame(dados_consolidados)

    # Salvar Dashboard Executivo (Excel)
    dashboard_excel_path = os.path.join(PASTA_TRABALHO, "dashboard", "DASHBOARD_MASTER_EXECUTIVO.xlsx")
    df_final.to_excel(dashboard_excel_path, index=False, engine="openpyxl")
    print(f"\n💾 Dashboard Executivo (XLSX) salvo em: {dashboard_excel_path}")

    # Salvar Dados Consolidados (CSV e JSON)
    dados_csv_path = os.path.join(PASTA_TRABALHO, "dashboard", "dados_consolidados.csv")
    df_final.to_csv(dados_csv_path, index=False, encoding="utf-8")
    print(f"💾 Dados Consolidados (CSV) salvo em: {dados_csv_path}")

    dados_json_path = os.path.join(PASTA_TRABALHO, "dashboard", "dados_detalhados.json")
    with open(dados_json_path, "w", encoding="utf-8") as f:
        json.dump(dados_consolidados, f, indent=2, ensure_ascii=False)
    print(f"💾 Dados Detalhados (JSON) salvo em: {dados_json_path}")

    # Geração de Dashboard Interativo (HTML - Exemplo simples)
    # Para um dashboard interativo real, seria necessário uma biblioteca como Plotly ou Dash
    dashboard_html_path = os.path.join(PASTA_TRABALHO, "dashboard", "dashboard_interativo.html")
    with open(dashboard_html_path, "w", encoding="utf-8") as f:
        f.write("<html><body><h1>Dashboard Interativo (Placeholder)</h1><p>Seu dashboard interativo real seria gerado aqui com bibliotecas como Plotly ou Dash.</p></body></html>")
    print(f"💾 Dashboard Interativo (HTML) salvo em: {dashboard_html_path}")

    # Geração do Blueprint Estratégico (PDF - Exemplo simples)
    pdf = PDF()
    pdf.add_page()
    pdf.chapter_title("BLUEPRINT ESTRATÉGICO FINAL")
    pdf.chapter_body("Este é o seu blueprint estratégico final, consolidando todos os insights.")
    pdf.chapter_body(f"Total de vídeos analisados: {len(df_final)}")
    pdf.chapter_body(f"Média de duração dos vídeos: {df_final["duracao_segundos"] .mean():.2f} segundos")
    pdf_blueprint_path = os.path.join(PASTA_TRABALHO, "blueprint", "BLUEPRINT_ESTRATEGICO_FINAL.pdf")
    pdf.output(pdf_blueprint_path)
    print(f"💾 Blueprint Estratégico (PDF) salvo em: {pdf_blueprint_path}")

    # Atualizar status no config
    config_path = os.path.join(PASTA_TRABALHO, "config", "config.json")
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)

    config["status_etapas"]["blueprint"] = True

    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    print("\n✅ GERAÇÃO DO BLUEPRINT FINAL E DASHBOARD CONCLUÍDA!")
    print("Todos os relatórios e o dashboard foram gerados com sucesso.")
    print("\n🎉 PROCESSO DE ENGENHARIA REVERSA CONCLUÍDO COM SUCESSO! 🎉")

# Executar geração de blueprint e dashboard
try:
    gerar_blueprint_dashboard()
except Exception as e:
    print(f"\n❌ ERRO GERAL NA GERAÇÃO DO BLUEPRINT E DASHBOARD: {e}")
    print("Por favor, corrija o erro acima antes de prosseguir.")


💾 Dashboard Executivo (XLSX) salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dashboard/DASHBOARD_MASTER_EXECUTIVO.xlsx
💾 Dados Consolidados (CSV) salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dashboard/dados_consolidados.csv
💾 Dados Detalhados (JSON) salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dashboard/dados_detalhados.json
💾 Dashboard Interativo (HTML) salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/dashboard/dashboard_interativo.html
💾 Blueprint Estratégico (PDF) salvo em: /content/drive/MyDrive/Videos Dona Done/_engenharia_reversa/blueprint/BLUEPRINT_ESTRATEGICO_FINAL.pdf

✅ GERAÇÃO DO BLUEPRINT FINAL E DASHBOARD CONCLUÍDA!
Todos os relatórios e o dashboard foram gerados com sucesso.

🎉 PROCESSO DE ENGENHARIA REVERSA CONCLUÍDO COM SUCESSO! 🎉
